<a href="https://colab.research.google.com/github/menon92/DL-Sneak-Peek/blob/master/%E0%A6%9F%E0%A7%87%E0%A6%A8%E0%A7%8D%E0%A6%B8%E0%A6%B0%E0%A6%AB%E0%A7%8D%E0%A6%B2%E0%A7%8B_%E0%A7%A8_%E0%A7%A6_%E0%A6%93_%E0%A6%95%E0%A7%87%E0%A6%B0%E0%A6%BE%E0%A6%B8_%E0%A6%AA%E0%A6%B0%E0%A6%BF%E0%A6%9A%E0%A6%BF%E0%A6%A4%E0%A6%BF_%E0%A6%AA%E0%A6%B0%E0%A7%8D%E0%A6%AC_%E0%A7%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
try:
  # %tensorflow_version এই কমান্ড কেবলমাত্র colab এ কাজ করে লোকাল নোটবুকে কাজ করবে না । 
  # এই জন্য try except ব্যাবহার করা হয়েছে । 
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

In [0]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [0]:
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

### ডাটাসেট ডাউনলোড 

ডাটাসেট ডাউনলোড করার জন্য আমরা `tf.keras.utils.get_file` এই ফাংশন ব্যাবহার করব । এখানে আমরা ডাটাসেট url এবং কি নামের (`fname='flower_photos'`) ফোল্ডারে ডাউনলোড হয়ে জমা হবে সেটা বলে দিচ্ছি । এবং `flower_photos.tgz` যেহেতু একটা জিপ ফাইল তাই এটাকে আনজিপ করার জন্য আমরা `untar=True` দিয়ে দিব । 

In [5]:
import pathlib

DATASET_URL = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'

data_dir = tf.keras.utils.get_file(origin=DATASET_URL, fname='flower_photos', untar=True)
data_dir = pathlib.Path(data_dir)
print('Dataset directory:', data_dir)

228818944/228813984 [==============================] - 2s 0us/step
Dataset directory: /root/.keras/datasets/flower_photos


In [12]:
FLOWERS = ['daisy', 'dandelion', 'roses','sunflowers','tulips']
for flower in FLOWERS:
    total_flower = len(list(data_dir.glob(flower + "/*.jpg")))
    print("`{:10s}` folder contain {} flower images".format(flower, total_flower))

`daisy     ` folder contain 633 flower images
`dandelion ` folder contain 898 flower images
`roses     ` folder contain 641 flower images
`sunflowers` folder contain 699 flower images
`tulips    ` folder contain 799 flower images


Keras এর ImageDataGenerator দিয়ে ডাটা পাইপলাইন লিখে ফেলি । যেহেতু আমি আগের পর্ব গুলোতে এইগুলো সম্পর্কে ডিটেইল বলেছি এই জন্য এখানে আমি আর এটা সম্পর্কে ব্যাখ্যা করব না । 

In [16]:
image_count = len(list(data_dir.glob('*/*.jpg')))

# defins params
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)

# define class names
CLASS_NAMES = np.array(
    [item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])

# init image data generator object 
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_data_gen = image_generator.flow_from_directory(
    directory=str(data_dir),
    batch_size=BATCH_SIZE,
    color_mode='rgb', # one of "grayscale", "rgb", "rgba". Default: "rgb". 
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    classes = list(CLASS_NAMES),
    class_mode='categorical' # "categorical", "binary", "sparse", "input", or None. Default: "categorical".
)

Found 3670 images belonging to 5 classes.


এখন আমরা tf.data দিয়ে পাইপলাইন লিখেফেলি

In [0]:
def get_label(file_path):
    # convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # print(parts)
    # The second to last is the class-directory
    return parts[-2] == CLASS_NAMES

def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

def process_path(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
    # ছোট ডাটাসেটের জন্য `ds = ds.cache(cache)` দরকার নাই । কিন্তু ডাটাসেট যদি 
    # অনেক বড় হয় তাহলে RAM এ সব ডাটা একসাথে ধরবে না । তখন আমাদের কে HDD 
    # এর সাহায্য নিতে হবে । 
    if cache:
        # যদি ডাটা অনেক বড় হয় তাহলে তাহলে আমরা ডাটাকে মেমরিতে না রেখে 
        # আমরা যে cache=যে পাথ দিব সেখানে ডাটা ক্যাশ করে রাখবে । 
        # নিচে এটারও আমরা একটা উদাহরণ দেখব । 
        if isinstance(cache, str):
            # HDD এ ডাটা ক্যাশ করে রাখবে । 
            ds = ds.cache(cache)
        else:
            # RAM এ ডাটা ক্যাশ করে রাখবে 
            ds = ds.cache()

    # ডাটা আমরা সাফল করে নিচ্ছি 
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    
    # ডাটাসেট অসংখ্যবার রিপিট হবে সেটা বলে দিলাম 
    ds = ds.repeat()

    # BATCH_SIZE অনুসারে আমারা ডাটা নিয়ে নিচ্ছি 
    ds = ds.batch(BATCH_SIZE)

    # যখন মডেল একটা ব্যাচ ডাটার উপর ট্রেনিং হতে থাকবে তখন পরবর্তী ব্যাচ আগে থেকেই প্রস্তুত করে রাখব 
    # যাতে করে মডেলের যখন নতুন ব্যাচ ডাটা দরকার তখন এসে অপেক্ষে করা না লাগে । 
    ds = ds.prefetch(buffer_size=AUTOTUNE)

    return ds

list_ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))
labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
train_ds = prepare_for_training(labeled_ds)

## সময় কোনটাতে কেমন লাগে সেটা দেখি

এখানে আমরা দেখব কোনটাতে কেমন সময় লাগে । প্রথমে আমরা একটা ফাংশন লিখি যেটা আমাদের কে এই ডাটাসেটের উপর কোনটা কোনটা কেমন সময় লাগতেছে সেটা হিসাব করে দিবে । 

In [0]:
import time
default_timeit_steps = 1000

def timeit(ds, steps=default_timeit_steps):
    start = time.time()
    it = iter(ds)
    for i in range(steps):
        batch = next(it)
        if i%10 == 0:
            print('.',end='')
    print()
    end = time.time()

    duration = end-start
    print("{} batches: {} s".format(steps, duration))
    print("{:0.5f} Images/s".format(BATCH_SIZE*steps/duration))

এখন আমরা উপরের দুইটা ডাটা জেনারেটরে মধ্যে সময় হিসাব করি । 

In [24]:
# `keras.preprocessing`
timeit(train_data_gen)

....................................................................................................
1000 batches: 84.32222032546997 s
379.49665 Images/s


In [25]:
# `tf.data`
timeit(train_ds)

....................................................................................................
1000 batches: 12.434528350830078 s
2573.47919 Images/s


এখানে আমরা দেখতে পাচ্ছি দুইটার মধ্যে কত পার্থক্য । কিন্তু আমাদের মেমরি যদি কম হয় তাহলে এটা আমরা করতে পারব না । 

একবার প্রসেস করা ডাটা আমরা `.cache` এর মাধ্যমে জমা করে রাখব । পরের বার যখন একই ডাটার জন্য রিকুয়েস্ট করবে তখন আগে দেখবে ডাটা টা ক্যাশ এ আছে কি না । যদি ক্যাশ এ থাকে তাহলে সেটা রিটার্ন করবে । যদি ক্যাশ এ না থাকে তাহলে ডাটা কে প্রসেস করে ক্যাশ জমা রাখবে । যাতে পরবর্তী সময়ে সেটা ব্যাবহার করা যায় । 

ক্যাশ ব্যাবহার না করে দেখি কেমন সময় লাগে । 

In [26]:
uncached_ds = prepare_for_training(labeled_ds, cache=False)
timeit(uncached_ds)

....................................................................................................
1000 batches: 58.00709366798401 s
551.65667 Images/s


যদি ডাটাসেট মেমরি তে ফিট না করে তাহলে আমরা ক্যাশ ডাটা কে একটা ফাইলে রাইট করে রাখব । এটা RAM এর মত ফাস্ট না হলেও `keras.preprocessing` থেকে অনেক ফাস্ট হবে । 

In [27]:
# create a caches directory
!mkdir caches

filecache_ds = prepare_for_training(labeled_ds, cache="caches/flowers.tfcache")
timeit(filecache_ds)

....................................................................................................
1000 batches: 37.12383055686951 s
861.98001 Images/s


আমরা `ls` করলে `flowers.tfcache` ফাইল দেখতে পারব । আমাদের ডাটাসেট যত বড় হবে এই ফাইলের সাইজ তত বড় হতে থাকবে ।


In [28]:
!du -sh caches/
print('*'*40)
!ls -l caches/

2.1G	caches/
****************************************
total 2158220
-rw-r--r-- 1 root root 2209769390 May 16 04:00 flowers.tfcache.data-00000-of-00001
-rw-r--r-- 1 root root     239989 May 16 04:00 flowers.tfcache.index


- মুল লেখা <a href="https://www.tensorflow.org/tutorials/load_data/images"> Image data loading </a>
- <a href="https://youtu.be/kVEOCfBy9uY?list=PLQY2H8rRoyvzIuB8rZXs7pfyjiSUs8Vza"> TF inut pipeline  </a>
- <a href="https://medium.com/analytics-vidhya/tensorflow-2-0-tf-data-api-eaa9889186cc"> TensorFlow 2.0: tf.data API</a>